In [1]:
import os
import sys
from pathlib import Path

if "workding_dir" not in locals():
    workding_dir = str(Path.cwd().parent)
os.chdir(workding_dir)
sys.path.append(workding_dir)
print("working dir:", workding_dir)

working dir: /home/inflaton/code/CrediNews


In [2]:
from dotenv import find_dotenv, load_dotenv

found_dotenv = find_dotenv(".env")

if len(found_dotenv) == 0:
    found_dotenv = find_dotenv(".env.example")
print(f"loading env vars from: {found_dotenv}")
load_dotenv(found_dotenv, override=True)

loading env vars from: /home/inflaton/code/CrediNews/.env


True

In [3]:
# !pip install wandb
import os
os.getenv("WANDB_API_KEY", "empty")[-5:]

'f541b'

In [4]:
import os
from transformers import DataCollatorWithPadding, AutoTokenizer, AutoModelForSequenceClassification

# Change these as needed.
model_name = "distilbert/distilbert-base-uncased"

# Triggers tokenizer download to cache
tokenizer = AutoTokenizer.from_pretrained(model_name)
print("Downloading and caching pre-trained model")

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Triggers model download to cache
AutoModelForSequenceClassification.from_pretrained(model_name)

2025-02-22 14:24:40.715277: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-22 14:24:40.723598: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740205480.733634 3358873 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740205480.736757 3358873 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-22 14:24:40.748190: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


## Training

With everything now downloaded and cached, we can now set up our training function. Our training function defines the training execution for a single hyperparameter configuration. For now we pull these hyperparameters from a config argument, but we’ll see later how this is passed in.

First we get our datasets- we only use the first half of the dev dataset for validation, and leave the rest of testing:

In [5]:
from datasets import load_dataset, concatenate_datasets, Dataset

datasets = load_dataset(
    "csv",
    data_files={
        "train": [
            "dataset/train_data_1.csv",
            "dataset/train_data_2.csv",
            "dataset/train_data_3.csv",
            "dataset/train_data_4.csv",
        ],
        "test": "dataset/test_data.csv",
        "rewritten_train": [
            "dataset/rewritten_train_data_1.csv",
            "dataset/rewritten_train_data_2.csv",
            "dataset/rewritten_train_data_3.csv",
            "dataset/rewritten_train_data_4.csv",
        ],
        "rewritten_test": "dataset/rewritten_test_data.csv",
    },
)

# Tokenize the dataset
def tokenize_function(example):
    return tokenizer(example["processed_full_content"], padding="max_length", truncation=True)

tokenized_datasets = datasets.map(tokenize_function, batched=True)

def get_datasets(config="original"):
    if config == "rewritten":
        train_dataset = tokenized_datasets["rewritten_train"]
        eval_dataset = tokenized_datasets["rewritten_test"]
    elif config == "original":
        train_dataset = tokenized_datasets["train"]
        eval_dataset = tokenized_datasets["test"]
    else:
        train_dataset = concatenate_datasets(
            [tokenized_datasets["train"], tokenized_datasets["rewritten_train"]]
        )
        eval_dataset = concatenate_datasets(
            [tokenized_datasets["test"], tokenized_datasets["rewritten_test"]]
        )

    return train_dataset, eval_dataset

Map:   0%|          | 0/54441 [00:00<?, ? examples/s]

Map:   0%|          | 0/6050 [00:00<?, ? examples/s]

Map:   0%|          | 0/54441 [00:00<?, ? examples/s]

Map:   0%|          | 0/6050 [00:00<?, ? examples/s]

In [6]:
get_datasets(config="original"), get_datasets(config="rewritten"), get_datasets(config="all")

((Dataset({
      features: ['label', 'full_content', 'processed_full_content', 'input_ids', 'attention_mask'],
      num_rows: 54441
  }),
  Dataset({
      features: ['label', 'full_content', 'processed_full_content', 'input_ids', 'attention_mask'],
      num_rows: 6050
  })),
 (Dataset({
      features: ['label', 'full_content', 'processed_full_content', 'input_ids', 'attention_mask'],
      num_rows: 54441
  }),
  Dataset({
      features: ['label', 'full_content', 'processed_full_content', 'input_ids', 'attention_mask'],
      num_rows: 6050
  })),
 (Dataset({
      features: ['label', 'full_content', 'processed_full_content', 'input_ids', 'attention_mask'],
      num_rows: 108882
  }),
  Dataset({
      features: ['label', 'full_content', 'processed_full_content', 'input_ids', 'attention_mask'],
      num_rows: 12100
  })))

In [7]:
def wandb_hp_space(trial):
    return {
        "method": "random",
        "metric": {"name": "accuracy", "goal": "maximize"},
        "parameters": {
            "learning_rate": {"distribution": "uniform", "min": 1e-6, "max": 1e-4},
            "per_device_train_batch_size": {"values": [16, 32]},
        },
    }

In [8]:
from transformers import AutoModelForSequenceClassification
import numpy as np
import evaluate
from transformers import EvalPrediction

# Load the accuracy metric from the evaluate library
accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred: EvalPrediction) -> dict:
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    return accuracy

def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)


In [9]:
from transformers import Trainer, TrainingArguments

def do_grid_search(config):
    train_dataset, eval_dataset = get_datasets(config)
    training_args = TrainingArguments(
        output_dir="./results",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=4,
        weight_decay=0.01,
    )

    trainer = Trainer(
        model_init=model_init,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )

    return trainer.hyperparameter_search(
        direction="maximize",
        backend="wandb", # or "ray", "optuna", "sigopt"
        hp_space=wandb_hp_space, #ray_hp_space, 
        n_trials=5,
    )

In [10]:
%%time

best_trial = do_grid_search("original")
best_trial

/home/inflaton/miniconda3/envs/fake-news/lib/python3.11/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_3358873/1062649897.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Create sweep with ID: k78ubaon
Sweep URL: https://wandb.ai/inflaton-ai/uncategorized/sweeps/k78ubaon


wandb: Agent Starting Run: kow0by3e with config:
wandb: 	learning_rate: 6.537781141095448e-06
wandb: 	per_device_train_batch_size: 16
wandb: Currently logged in as: inflaton-sg (inflaton-ai) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Trying to set _wandb in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set assignments in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set metric in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


wandb: WARNING Config item 'per_device_train_batch_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).


...

wandb: WARNING Config item 'per_device_train_batch_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).


Epoch,Training Loss,Validation Loss,Accuracy
1,0.045800,0.072953,0.984628
2,0.022600,0.027620,0.991570
3,0.008400,0.033815,0.992231
4,0.002700,0.043594,0.992727


eval/accuracy,▁▇██
eval/loss,█▁▂▃
eval/runtime,▄▇▁█
eval/samples_per_second,▅▂█▁
eval/steps_per_second,▅▂█▁
train/epoch,▁▂▂▂▃▃▄▄▄▅▅▆▆▇▇███
train/global_step,▁▂▂▂▃▃▄▄▄▅▅▆▆▇▇███
train/grad_norm,▁▄█▂▁▁▂▁▁▁▂▁▁
train/learning_rate,█▇▇▆▆▅▅▄▃▃▂▂▁
train/loss,█▄▃▂▂▂▂▁▁▁▁▁▁
eval/accuracy,0.99273


CPU times: user 3h 11min 38s, sys: 1min 52s, total: 3h 13min 30s
Wall time: 3h 14min


BestRun(run_id='hcjvxp5p', objective=0.9948760330578512, hyperparameters={'learning_rate': 5.644497791637146e-05, 'per_device_train_batch_size': 16, 'assignments': {}, 'metric': 'eval/loss'}, run_summary=None)

In [11]:
%%time

best_trial_combined = do_grid_search("combined")
best_trial_combined

/home/inflaton/miniconda3/envs/fake-news/lib/python3.11/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_3358873/1062649897.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Create sweep with ID: 1pmhicln
Sweep URL: https://wandb.ai/inflaton-ai/uncategorized/sweeps/1pmhicln


wandb: Agent Starting Run: 93v46at6 with config:
wandb: 	learning_rate: 7.225528979449591e-05
wandb: 	per_device_train_batch_size: 32


Trying to set _wandb in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set assignments in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set metric in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


wandb: WARNING Config item 'per_device_train_batch_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).


...

wandb: WARNING Config item 'per_device_train_batch_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).


Epoch,Training Loss,Validation Loss,Accuracy
1,0.123300,0.108198,0.961901
2,0.081600,0.130055,0.962893
3,0.049500,0.159044,0.963884
4,0.019800,0.189388,0.964132


eval/accuracy,▁▄▇█
eval/loss,▁▃▅█
eval/runtime,▂█▁█
eval/samples_per_second,▇▁█▁
eval/steps_per_second,▇▁█▁
train/epoch,▁▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇█████
train/global_step,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇████
train/grad_norm,▂▁▁▂▁▂▁▁▁▂▂▁▁▁▁▁▃▁▁▁▁▂▁▁▂▃▁▁▁▁▁▁▁▁▁▁▁▁█▁
train/learning_rate,████▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁
train/loss,█▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▂▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
eval/accuracy,0.96413


CPU times: user 6h 26min 23s, sys: 3min 22s, total: 6h 29min 45s
Wall time: 6h 30min 10s


BestRun(run_id='a3vo985i', objective=0.9652066115702479, hyperparameters={'learning_rate': 5.065523986980171e-05, 'per_device_train_batch_size': 32, 'assignments': {}, 'metric': 'eval/loss'}, run_summary=None)

In [12]:
%%time

best_trial_rewritten = do_grid_search("rewritten")
best_trial_rewritten

/home/inflaton/miniconda3/envs/fake-news/lib/python3.11/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_3358873/1062649897.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Create sweep with ID: uyvcwcf5
Sweep URL: https://wandb.ai/inflaton-ai/uncategorized/sweeps/uyvcwcf5


wandb: Agent Starting Run: xflumwy4 with config:
wandb: 	learning_rate: 7.481087570424631e-05
wandb: 	per_device_train_batch_size: 16


Trying to set _wandb in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set assignments in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set metric in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


wandb: WARNING Config item 'per_device_train_batch_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).


...

wandb: WARNING Config item 'per_device_train_batch_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).


Epoch,Training Loss,Validation Loss,Accuracy
1,0.254300,0.262873,0.919008
2,0.207000,0.216914,0.918017
3,0.141300,0.256720,0.927934
4,0.107000,0.232754,0.928926


eval/accuracy,▂▁▇█
eval/loss,█▁▇▃
eval/runtime,█▆▄▁
eval/samples_per_second,▁▃▅█
eval/steps_per_second,▁▃▅█
train/epoch,▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▆▆▆▆▆▇▇▇▇████
train/global_step,▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▆▆▆▆▆▇▇▇▇████
train/grad_norm,▁▂▂▁▁▁▂█▁▁▁▂▂▁▆▁▂▁▁▁▂▂▁▁▁▁▂
train/learning_rate,██▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▃▃▃▃▂▂▂▂▁▁
train/loss,█▆▆▅▅▅▅▄▄▄▄▃▄▃▂▂▂▂▂▂▁▁▁▁▁▁▁
eval/accuracy,0.92893


CPU times: user 3h 12min 20s, sys: 2min 11s, total: 3h 14min 32s
Wall time: 3h 15min 8s


BestRun(run_id='6iv4aycl', objective=0.9375206611570248, hyperparameters={'learning_rate': 6.490693450061599e-05, 'per_device_train_batch_size': 32, 'assignments': {}, 'metric': 'eval/loss'}, run_summary=None)